In [8]:
import cv2
import numpy as np
import os

P = np.array([2/3, 1/3])
H = -np.sum(P*np.log2(P))
print(-np.log2(P))
print(f'H = {H:g} bits')

P2 = np.array([0.533, 0.133, 0.133, 0.2])
H2 = -np.sum(P2*np.log2(P2))/2
print(f'H2 = {H2:g} bits/symbol')

P3 = np.array([0.9,0.1])
H3 = -np.sum(P3*np.log2(P3))
print(-np.log2(P3))
print(f'H3 = {H3:g} bits')

[0.5849625 1.5849625]
H = 0.918296 bits
H2 = 0.861216 bits/symbol
[0.15200309 3.32192809]
H3 = 0.468996 bits


#### (1) Arithmetic Code

In [ ]:
def encode(lb, ub, p1, b):
    if b==1:
        ub = lb + (ub-lb)*p1
    else:
        lb = lb + (ub-lb)*p1
    return lb, ub

s = [1,1,0,1,1,0,0,1,1,1]

lb, ub = 0, 1
for n in range(10):
    lb, ub = encode(lb, ub, 0.7, s[n])
    print('{:0.6f}, {:0.6f}'.format(lb, ub))

cw, lb0, ub0 = 0, lb, ub

for n in range(20):
    k = 1/2**n
    if k < ub:
        cw += k
        lb -= k
        ub -= k
        print(n, k, '{:0.6f}, {:0.6f}'.format(lb, ub))
        if cw >= lb0:
            break
print(cw)

0.000000, 0.700000
0.000000, 0.490000
0.343000, 0.490000
0.343000, 0.445900
0.343000, 0.415030
0.393421, 0.415030
0.408547, 0.415030
0.408547, 0.413085
0.408547, 0.411724
0.408547, 0.410771
2 0.25 0.158547, 0.160771
3 0.125 0.033547, 0.035771
5 0.03125 0.002297, 0.004521
8 0.00390625 -0.001609, 0.000615
0.41015625


In [ ]:
import numpy as np
s = np.array([0,0,1,0,0,1,1,0,0,0])
p0 = 0.7
lb = 0
ub = 1
for n in range(10):
    step = ub - lb
    thrs = lb + step*p0
    if s[n] == 1:   
        ub = thrs
    else:
        lb = thrs

# find the combination of binary fractions
bf = 0.5
cw = []
while True:
    if lb <= bf < ub: # found
        cw.append(1)
        break
    elif lb > bf:
        cw.append(1)
        lb -= bf
        ub -= bf
        bf /= 2
    else:
        cw.append(0)
        bf /= 2

print(cw)

#### (2) PNG

In [20]:
import cv2
import os

filename = "../data/jaguar.bmp" #"../data/lena.png"
img = cv2.imread(filename) 
size0 = os.path.getsize(filename)
print('BMP : Saved using {} kbytes'.format(size0/1000))

cv2.imwrite('img_compressed.tiff', img)
size1 = os.path.getsize('img_compressed.tiff')
print('TIFF: {} kbytes'.format(size1/1000), end=' ')
compressed = cv2.imread("img_compressed.tiff")
psnr = cv2.PSNR(img,compressed)
print(f'\nPSNR = {psnr:0.2f} dB')

cv2.imshow('tiff', compressed)
cv2.waitKey()
cv2.destroyAllWindows()

for n in range(10):
    cv2.imwrite('img_compressed.png', img, [cv2.IMWRITE_PNG_COMPRESSION, 1])
    # Complexity: 0~9 (low cmplx and fast decoding <-> high cmplx and slow decoding)
    size1 = os.path.getsize('img_compressed.png')
    print('PNG {}: {} kbytes'.format(n, size1/1000))

compressed = cv2.imread("img_compressed.png")
psnr = cv2.PSNR(img,compressed)
print(f'\nPSNR = {psnr:0.2f} dB')

BMP : Saved using 6912.054 kbytes
TIFF: 2425.898 kbytes 
PSNR = 361.20 dB
PNG 0: 2032.064 kbytes
PNG 1: 2032.064 kbytes
PNG 2: 2032.064 kbytes
PNG 3: 2032.064 kbytes
PNG 4: 2032.064 kbytes
PNG 5: 2032.064 kbytes
PNG 6: 2032.064 kbytes
PNG 7: 2032.064 kbytes
PNG 8: 2032.064 kbytes
PNG 9: 2032.064 kbytes

PSNR = 361.20 dB
